# AULA 3 DEMONSTRAÇÃO 3- Introdução à Otimização para Redes Neurais

Aula 3- Aula Assíncrona
https://www.youtube.com/watch?v=O3h4kUdSV4k&t=2s&ab_channel=MoacirAntonelliPonti

---

## Otimização para Redes Neurais

- Redes Densas em pytorch e detalhes:
    - camadas pré-implementadas
    - opções para acessar parametros
    - uso do gradiente
    - backpropagation e algoritmos de otimização
    
---

In [68]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

### Projeto Rede Densa

- exemplo da aula teórica
    - entrada: imagens 28x28 pixels
    - saída: 10 classes (dígitos de 0 a 9)

In [69]:
class Network(nn.Module): # herda uma classe com diversos parametros incapsulados

    def __init__(self):
        # inicia conforme a superclasse nn.Module
        super(Network, self).__init__()
        # cria o projeto da rede neural
        # duas camadas Fully Connected, fc1 = hidden e fc2 = final
        self.fc1 = nn.Linear(784, 32) # entrada = 784, saída = 32
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # achatar todas as camadas, exceto uma
        x = torch.flatten(x, 1) # recebe minibatches
        x = F.relu(self.fc1(x)) # camada densa + relu
        x = self.fc2(x) # camada densa linear (sem softmax por enquanto)
        return x

In [70]:
net = Network()
net

Network(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [71]:
input_random = torch.randn(1, 1, 28, 28) # 1 batche com 1 canal de cor 28x28 = imagem aleatoria

output = net(input_random)
output

tensor([[-0.1218, -0.2955,  0.1731,  0.3342, -0.1597,  0.1669, -0.6116,  0.2721,
          0.5229,  0.0752]], grad_fn=<AddmmBackward0>)

- acessando os parametros da rede

In [72]:
params = list(net.parameters())
print(len(params))
# primeira camada
print(params[0].size())
print(params[0])

4
torch.Size([32, 784])
Parameter containing:
tensor([[-0.0025,  0.0247, -0.0167,  ..., -0.0240,  0.0194,  0.0100],
        [-0.0108,  0.0128, -0.0150,  ...,  0.0055,  0.0135,  0.0044],
        [ 0.0297,  0.0067, -0.0187,  ..., -0.0187,  0.0133, -0.0235],
        ...,
        [-0.0221,  0.0147,  0.0319,  ..., -0.0031, -0.0312,  0.0209],
        [-0.0303,  0.0185, -0.0327,  ..., -0.0153, -0.0073,  0.0348],
        [-0.0027,  0.0277,  0.0099,  ..., -0.0025, -0.0236, -0.0162]],
       requires_grad=True)


### Backpropagation

* já está pronto: `backward`

In [73]:
# zerar o buffer de gradientes
net.zero_grad()
output.backward(torch.randn(1,10)) # aleatorio ´so para exemplo

### Rede operando

In [74]:
output = net(input_random)
target = torch.randn(10) # target aleatorio como exemplo
print(target)
target = target.view(1, -1) # um elemento em uma dimensão e oresto em outra
print(target)

criterion = nn.MSELoss() # mean sqaured error
loss = criterion(output, target)
print(loss)

tensor([-1.5134,  0.1599,  1.6754,  0.0374, -1.7905,  1.2918,  0.6488,  1.4015,
         1.1718, -0.6582])
tensor([[-1.5134,  0.1599,  1.6754,  0.0374, -1.7905,  1.2918,  0.6488,  1.4015,
          1.1718, -0.6582]])
tensor(1.2237, grad_fn=<MseLossBackward0>)


### Backpropagation com `backward`

In [75]:
net.zero_grad()
# antes
print('antes', net.fc1.bias.grad)

loss.backward()
print('depois', net.fc1.bias.grad)

antes tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
depois tensor([-0.0395,  0.0000,  0.0000, -0.0856,  0.0000,  0.0000, -0.0344,  0.0000,
         0.0928,  0.0349, -0.0931, -0.0774,  0.0789,  0.0000, -0.0480,  0.0000,
         0.0000,  0.0000,  0.0000, -0.0660, -0.0531,  0.0000,  0.0445, -0.0205,
        -0.0853, -0.1023,  0.0000,  0.0136,  0.0000,  0.0000,  0.0000,  0.0074])


### Otimizadores

- adaptacao dos parametros da rede
    - otimziador: SGD, Adam, etc

In [76]:
print(params[0][0][:10])

tensor([-0.0025,  0.0247, -0.0167,  0.0242, -0.0306, -0.0105, -0.0202,  0.0215,
        -0.0039, -0.0324], grad_fn=<SliceBackward0>)


In [77]:
import torch.optim as optim

# objeto otimizador
optimizer = optim.SGD(net.parameters(), lr=0.01)

# para cada loop de treinamento

# zerar o buffer dos grads
optimizer.zero_grad()
# gerar a saida e computar gradientes com relacao a func de perda
output = net(input_random)
loss = criterion(output, target)
loss.backward()
# adaptar pesos
optimizer.step()

In [78]:
print(params[0][0][:10])

tensor([-0.0015,  0.0250, -0.0165,  0.0241, -0.0309, -0.0103, -0.0197,  0.0212,
        -0.0040, -0.0312], grad_fn=<SliceBackward0>)
